<a href="https://www.kaggle.com/code/tahmidamajumder/3-machine-learning?scriptVersionId=215357572" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

import graphviz
from sklearn import tree

# remove warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Loading the data
new_data = pd.read_csv(r"/kaggle/input/new-data-2015-2024/new_data_2015_2024.csv")

# print the shape
print(new_data.shape)

#runs the first 5 rows
new_data.head()

(1164300, 22)


,Unnamed: 0,Date,Country,City,co,no2,o3,pm10,pm25,so2,...,temperature,CO_SubIndex,NO2_SubIndex,O3_SubIndex,PM10_SubIndex,PM2.5_SubIndex,SO2_SubIndex,Checks,AQI,AQI_bucket
0,0,2014-12-29,Austria,Graz,51.0,204.0,NaN,226.0,NaN,60.0,...,NaN,500.002496,119.884170,0,135.858586,0.000000,80.769231,4,500.0,Hazardous
1,1,2014-12-29,Austria,Innsbruck,18.0,95.0,NaN,80.0,NaN,14.0,...,NaN,216.778523,94.565217,0,62.626263,0.000000,20.000000,4,217.0,Very Unhealthy
2,2,2014-12-29,Austria,Linz,82.0,188.0,NaN,176.0,137.0,77.0,...,NaN,500.157264,116.795367,0,110.606061,211.511512,100.458716,5,500.0,Hazardous
3,3,2014-12-29,Austria,Salzburg,56.0,90.0,NaN,80.0,NaN,35.0,...,NaN,500.027459,89.130435,0,62.626263,0.000000,50.000000,4,500.0,Hazardous
4,4,2014-12-29,Austria,Vienna,40.0,387.0,NaN,357.0,124.0,83.0,...,NaN,395.477387,154.513889,0,202.898551,198.998569,103.211009,5,395.0,Hazardous


In [3]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164300 entries, 0 to 1164299
Data columns (total 22 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1164300 non-null  int64  
 1   Date            1164300 non-null  object 
 2   Country         1164300 non-null  object 
 3   City            1164300 non-null  object 
 4   co              875003 non-null   float64
 5   no2             1124700 non-null  float64
 6   o3              1021257 non-null  float64
 7   pm10            1077279 non-null  float64
 8   pm25            1044682 non-null  float64
 9   so2             948888 non-null   float64
 10  humidity        875577 non-null   float64
 11  pressure        871790 non-null   float64
 12  temperature     876097 non-null   float64
 13  CO_SubIndex     1164300 non-null  float64
 14  NO2_SubIndex    1164300 non-null  float64
 15  O3_SubIndex     1164300 non-null  int64  
 16  PM10_SubIndex   1164300 non-null  fl

In [4]:
new_data['Country_City'] = new_data['Country'] + '_' + new_data['City']

In [5]:
new_data.head()

,Unnamed: 0,Date,Country,City,co,no2,o3,pm10,pm25,so2,...,CO_SubIndex,NO2_SubIndex,O3_SubIndex,PM10_SubIndex,PM2.5_SubIndex,SO2_SubIndex,Checks,AQI,AQI_bucket,Country_City
0,0,2014-12-29,Austria,Graz,51.0,204.0,NaN,226.0,NaN,60.0,...,500.002496,119.884170,0,135.858586,0.000000,80.769231,4,500.0,Hazardous,Austria_Graz
1,1,2014-12-29,Austria,Innsbruck,18.0,95.0,NaN,80.0,NaN,14.0,...,216.778523,94.565217,0,62.626263,0.000000,20.000000,4,217.0,Very Unhealthy,Austria_Innsbruck
2,2,2014-12-29,Austria,Linz,82.0,188.0,NaN,176.0,137.0,77.0,...,500.157264,116.795367,0,110.606061,211.511512,100.458716,5,500.0,Hazardous,Austria_Linz
3,3,2014-12-29,Austria,Salzburg,56.0,90.0,NaN,80.0,NaN,35.0,...,500.027459,89.130435,0,62.626263,0.000000,50.000000,4,500.0,Hazardous,Austria_Salzburg
4,4,2014-12-29,Austria,Vienna,40.0,387.0,NaN,357.0,124.0,83.0,...,395.477387,154.513889,0,202.898551,198.998569,103.211009,5,395.0,Hazardous,Austria_Vienna


In [6]:
spain_data = new_data[new_data['Country_City'].str.contains('Spain')]
print(spain_data['Country_City'].unique())

['Spain_Barcelona' 'Spain_Bilbao' 'Spain_Burgos'
 'Spain_Castelló de la Plana' 'Spain_Donostia / San Sebastián'
 'Spain_Gasteiz / Vitoria' 'Spain_Las Palmas de Gran Canaria'
 'Spain_Madrid' 'Spain_Murcia' 'Spain_Oviedo' 'Spain_Pamplona'
 'Spain_Salamanca' 'Spain_Santa Cruz de Tenerife' 'Spain_Santander'
 'Spain_Valencia' 'Spain_Córdoba' 'Spain_Granada' 'Spain_Málaga'
 'Spain_Sevilla' 'Spain_Huelva' 'Spain_Palma' 'Spain_Valladolid'
 'Spain_Zaragoza']


In [7]:
new_data['Country_City'] = new_data['Country_City'].replace('Spain_Donostia / San Sebastián', 'Spain_Donostia San Sebastián')
new_data['Country_City'] = new_data['Country_City'].replace('Spain_Gasteiz / Vitoria', 'Spain_Gasteiz Vitoria')

In [8]:
spain_data_1 = new_data[new_data['Country_City'].str.contains('Spain')]
print(spain_data_1['Country_City'].unique())

['Spain_Barcelona' 'Spain_Bilbao' 'Spain_Burgos'
 'Spain_Castelló de la Plana' 'Spain_Donostia San Sebastián'
 'Spain_Gasteiz Vitoria' 'Spain_Las Palmas de Gran Canaria' 'Spain_Madrid'
 'Spain_Murcia' 'Spain_Oviedo' 'Spain_Pamplona' 'Spain_Salamanca'
 'Spain_Santa Cruz de Tenerife' 'Spain_Santander' 'Spain_Valencia'
 'Spain_Córdoba' 'Spain_Granada' 'Spain_Málaga' 'Spain_Sevilla'
 'Spain_Huelva' 'Spain_Palma' 'Spain_Valladolid' 'Spain_Zaragoza']


In [9]:
# removing columns where missing values is more than 800,000
new_data = new_data.drop(['Unnamed: 0','Country', 'City'], axis=1)

In [10]:
new_data.columns

Index(['Date', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2', 'humidity',
       'pressure', 'temperature', 'CO_SubIndex', 'NO2_SubIndex', 'O3_SubIndex',
       'PM10_SubIndex', 'PM2.5_SubIndex', 'SO2_SubIndex', 'Checks', 'AQI',
       'AQI_bucket', 'Country_City'],
      dtype='object')

In [11]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

In [12]:
new_data['Date'] = new_data['Date'].apply(str_to_datetime)
new_data['Date']

0         2014-12-29
1         2014-12-29
2         2014-12-29
3         2014-12-29
4         2014-12-29
             ...    
1164295   2024-12-17
1164296   2024-12-17
1164297   2024-12-17
1164298   2024-12-17
1164299   2024-12-17
Name: Date, Length: 1164300, dtype: datetime64[ns]

In [13]:
new_data.head()

,Date,co,no2,o3,pm10,pm25,so2,humidity,pressure,temperature,CO_SubIndex,NO2_SubIndex,O3_SubIndex,PM10_SubIndex,PM2.5_SubIndex,SO2_SubIndex,Checks,AQI,AQI_bucket,Country_City
0,2014-12-29,51.0,204.0,NaN,226.0,NaN,60.0,NaN,NaN,NaN,500.002496,119.884170,0,135.858586,0.000000,80.769231,4,500.0,Hazardous,Austria_Graz
1,2014-12-29,18.0,95.0,NaN,80.0,NaN,14.0,NaN,NaN,NaN,216.778523,94.565217,0,62.626263,0.000000,20.000000,4,217.0,Very Unhealthy,Austria_Innsbruck
2,2014-12-29,82.0,188.0,NaN,176.0,137.0,77.0,NaN,NaN,NaN,500.157264,116.795367,0,110.606061,211.511512,100.458716,5,500.0,Hazardous,Austria_Linz
3,2014-12-29,56.0,90.0,NaN,80.0,NaN,35.0,NaN,NaN,NaN,500.027459,89.130435,0,62.626263,0.000000,50.000000,4,500.0,Hazardous,Austria_Salzburg
4,2014-12-29,40.0,387.0,NaN,357.0,124.0,83.0,NaN,NaN,NaN,395.477387,154.513889,0,202.898551,198.998569,103.211009,5,395.0,Hazardous,Austria_Vienna


In [14]:
# Group by the 'Country_City' column
grouped = new_data.groupby('Country_City')

# Save each group as a separate CSV file
for location, group in grouped:
    # filename based on the location
    location = location.strip() + '.csv'
    group.to_csv(location, index=False)
    print(f"Saved dataset for {location} as {location}")

Saved dataset for Argentina_Buenos Aires.csv as Argentina_Buenos Aires.csv
Saved dataset for Australia_Adelaide.csv as Australia_Adelaide.csv
Saved dataset for Australia_Brisbane.csv as Australia_Brisbane.csv
Saved dataset for Australia_Canberra.csv as Australia_Canberra.csv
Saved dataset for Australia_Darwin.csv as Australia_Darwin.csv
Saved dataset for Australia_Melbourne.csv as Australia_Melbourne.csv
Saved dataset for Australia_Newcastle.csv as Australia_Newcastle.csv
Saved dataset for Australia_Perth.csv as Australia_Perth.csv
Saved dataset for Australia_Sydney.csv as Australia_Sydney.csv
Saved dataset for Australia_Wollongong.csv as Australia_Wollongong.csv
Saved dataset for Austria_Graz.csv as Austria_Graz.csv
Saved dataset for Austria_Innsbruck.csv as Austria_Innsbruck.csv
Saved dataset for Austria_Linz.csv as Austria_Linz.csv
Saved dataset for Austria_Salzburg.csv as Austria_Salzburg.csv
Saved dataset for Austria_Vienna.csv as Austria_Vienna.csv
Saved dataset for Belgium_Antwe

In [15]:
# Compress all CSV files present in the working directory
!zip -r /kaggle/working/all_csvs.zip /kaggle/working/*.csv

  adding: kaggle/working/Argentina_Buenos Aires.csv (deflated 87%)
  adding: kaggle/working/Australia_Adelaide.csv (deflated 87%)
  adding: kaggle/working/Australia_Brisbane.csv (deflated 84%)
  adding: kaggle/working/Australia_Canberra.csv (deflated 87%)
  adding: kaggle/working/Australia_Darwin.csv (deflated 88%)
  adding: kaggle/working/Australia_Melbourne.csv (deflated 85%)
  adding: kaggle/working/Australia_Newcastle.csv (deflated 86%)
  adding: kaggle/working/Australia_Perth.csv (deflated 85%)
  adding: kaggle/working/Australia_Sydney.csv (deflated 84%)
  adding: kaggle/working/Australia_Wollongong.csv (deflated 87%)
  adding: kaggle/working/Austria_Graz.csv (deflated 86%)
  adding: kaggle/working/Austria_Innsbruck.csv (deflated 89%)
  adding: kaggle/working/Austria_Linz.csv (deflated 85%)
  adding: kaggle/working/Austria_Salzburg.csv (deflated 88%)
  adding: kaggle/working/Austria_Vienna.csv (deflated 83%)
  adding: kaggle/working/Belgium_Antwerpen.csv (deflated 84%)
  adding: k

# **4.Splitting & Preprocessing the Raw Data**

1. Working with one location for to check which model gives better result
   Argentina_Buenos Aires

In [16]:
data= pd.read_csv(r'/kaggle/input/location-separated-data/Argentina_Buenos Aires.csv')
print(data.shape)
data.head()

(2477, 20)


,Date,co,no2,o3,pm10,pm25,so2,humidity,pressure,temperature,CO_SubIndex,NO2_SubIndex,O3_SubIndex,PM10_SubIndex,PM2.5_SubIndex,SO2_SubIndex,Checks,AQI,AQI_bucket,Country_City
0,2015-04-27,3.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,34.090909,1.886792,0,2.777778,0.0,0.0,3,34.0,Good,Argentina_Buenos Aires
1,2015-04-28,61.0,45.0,NaN,65.0,NaN,NaN,NaN,NaN,NaN,500.052421,42.452830,0,55.050505,0.0,0.0,3,500.0,Hazardous,Argentina_Buenos Aires
2,2015-04-29,70.0,48.0,NaN,51.0,NaN,NaN,NaN,NaN,NaN,500.097354,45.283019,0,47.222222,0.0,0.0,3,500.0,Hazardous,Argentina_Buenos Aires
3,2015-05-03,68.0,45.0,NaN,56.0,NaN,NaN,NaN,NaN,NaN,500.087369,42.452830,0,50.505051,0.0,0.0,3,500.0,Hazardous,Argentina_Buenos Aires
4,2015-05-04,72.0,46.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,500.107339,43.396226,0,10.185185,0.0,0.0,3,500.0,Hazardous,Argentina_Buenos Aires


In [17]:
data = data.drop(['Checks', 'Country_City'],axis=1)

In [18]:
data = data.drop(['Date'],axis=1)

In [19]:
data.columns

Index(['co', 'no2', 'o3', 'pm10', 'pm25', 'so2', 'humidity', 'pressure',
       'temperature', 'CO_SubIndex', 'NO2_SubIndex', 'O3_SubIndex',
       'PM10_SubIndex', 'PM2.5_SubIndex', 'SO2_SubIndex', 'AQI', 'AQI_bucket'],
      dtype='object')

In [20]:
num = data.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
cat = data.select_dtypes(include=['object'])

print(num.columns)
print(cat.columns)

Index(['co', 'no2', 'o3', 'pm10', 'pm25', 'so2', 'humidity', 'pressure',
       'temperature', 'CO_SubIndex', 'NO2_SubIndex', 'O3_SubIndex',
       'PM10_SubIndex', 'PM2.5_SubIndex', 'SO2_SubIndex', 'AQI'],
      dtype='object')
Index(['AQI_bucket'], dtype='object')


In [21]:
X = data[['co', 'no2', 'o3', 'pm10', 'pm25', 'so2','CO_SubIndex', 'NO2_SubIndex', 'O3_SubIndex',
       'PM10_SubIndex', 'PM2.5_SubIndex', 'SO2_SubIndex', 'AQI_bucket']]

y = data['AQI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [22]:
# OneHotEncoding the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing numerical values with mean
            ('scaler', MinMaxScaler())
        ]), ['co', 'no2', 'o3', 'pm10', 'pm25', 'so2','CO_SubIndex', 'NO2_SubIndex', 'O3_SubIndex',
       'PM10_SubIndex', 'PM2.5_SubIndex', 'SO2_SubIndex']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['AQI_bucket'])  # Apply OneHotEncoding and handle unknown
    ])
# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# **5.Model Training & Testing**

In [23]:
# training with LinearRegression

lm = LinearRegression(fit_intercept = True)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', lm)])

# Fit the pipeline to the training data
pipeline.fit(X_train,y_train)

# Predict on the training and test datasets
y_pred_train_lm = pipeline.predict(X_train)
y_pred_test_lm = pipeline.predict(X_test)

In [24]:
# Model Accuracy on training and testing dataset

print('The r2 Accuracy on the training dataset prediction is: ',r2_score(y_train,y_pred_train_lm) )
print('The r2 Accuracy on the test dataset prediction is: ',r2_score(y_test,y_pred_test_lm) )

print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE  on the training dataset is: ',sqrt(mean_squared_error(y_train, y_pred_train_lm)))
print('The RMSE  on the testing dataset is: ',sqrt(mean_squared_error(y_test, y_pred_test_lm)))

print("")
# The Mean Absolute Error (MAE)
print('The MAE  on the training dataset is: ',mean_absolute_error(y_train, y_pred_train_lm))
print('The MAE  on the testing dataset is: ',mean_absolute_error(y_test, y_pred_test_lm))

The r2 Accuracy on the training dataset prediction is:  0.9962818747198151
The r2 Accuracy on the test dataset prediction is:  0.9962191630569589

The RMSE  on the training dataset is:  8.714656796717989
The RMSE  on the testing dataset is:  8.890766434778614

The MAE  on the training dataset is:  5.109403098473954
The MAE  on the testing dataset is:  5.263588160620774


In [25]:
# training with KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=2)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', neigh)])

# Fit the pipeline to the training data
pipeline.fit(X_train,y_train)

# Predict on the training and test datasets
y_pred_train_knn = pipeline.predict(X_train)
y_pred_test_knn = pipeline.predict(X_test)

In [26]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train,y_pred_train_knn))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_knn))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE  on the training dataset is: ',sqrt(mean_squared_error(y_train,y_pred_train_knn)))
print('The RMSE  on the testing dataset is: ',sqrt(mean_squared_error(y_test, y_pred_test_knn)))

print("")
# The Mean Absolute Error (MAE)
print('The MAE  on the training dataset is: ',mean_absolute_error(y_train,y_pred_train_knn))
print('The MAE  on the testing dataset is: ',mean_absolute_error(y_test, y_pred_test_knn))

The R2 score on the training dataset prediction is:  0.9960086275991438
The R2 score on the test dataset prediction is:  0.9871573257807041

The RMSE  on the training dataset is:  9.029202720439079
The RMSE  on the testing dataset is:  16.386000368134873

The MAE  on the training dataset is:  3.231701161029783
The MAE  on the testing dataset is:  6.455645161290323


In [27]:
# Training with random forest regressor
random = RandomForestRegressor()

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', random)])

# Fit the pipeline to the training data
pipeline.fit(X_train,y_train)

# Predict on the training and test datasets
y_pred_train_random = pipeline.predict(X_train)
y_pred_test_random = pipeline.predict(X_test)

In [28]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train,y_pred_train_random))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_random))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE  on the training dataset is: ',sqrt(mean_squared_error(y_train,y_pred_train_random)))
print('The RMSE  on the testing dataset is: ',sqrt(mean_squared_error(y_test, y_pred_test_random)))

print("")
# The Mean Absolute Error (MAE)
print('The MAE  on the training dataset is: ',mean_absolute_error(y_train,y_pred_train_random))
print('The MAE  on the testing dataset is: ',mean_absolute_error(y_test, y_pred_test_random))

The R2 score on the training dataset prediction is:  0.9999956591477333
The R2 score on the test dataset prediction is:  0.9999532898274501

The RMSE  on the training dataset is:  0.2977664625661815
The RMSE  on the testing dataset is:  0.9882139109883892

The MAE  on the training dataset is:  0.04043917213528535
The MAE  on the testing dataset is:  0.1361895161290325


In [29]:
# Training with Decision Tree regressor
dt = DecisionTreeRegressor()

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', dt)])

# Fit the pipeline to the training data
pipeline.fit(X_train,y_train)

# Predict on the training and test datasets
y_pred_train_dt = pipeline.predict(X_train)
y_pred_test_dt = pipeline.predict(X_test)

In [30]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train,y_pred_train_dt))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_dt))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE  on the training dataset is: ',sqrt(mean_squared_error(y_train,y_pred_train_dt)))
print('The RMSE  on the testing dataset is: ',sqrt(mean_squared_error(y_test, y_pred_test_dt)))

print("")
# The Mean Absolute Error (MAE)
print('The MAE  on the training dataset is: ',mean_absolute_error(y_train,y_pred_train_dt))
print('The MAE  on the testing dataset is: ',mean_absolute_error(y_test, y_pred_test_dt))

The R2 score on the training dataset prediction is:  1.0
The R2 score on the test dataset prediction is:  0.999967984082471

The RMSE  on the training dataset is:  0.0
The RMSE  on the testing dataset is:  0.8181410872885418

The MAE  on the training dataset is:  0.0
The MAE  on the testing dataset is:  0.09274193548387097


In [31]:
# Training with Support Vector Machine regressor
svr = SVR(kernel='linear')

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', svr)])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict on the training and test datasets
y_pred_train_svr = pipeline.predict(X_train)
y_pred_test_svr = pipeline.predict(X_test)

In [32]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train, y_pred_train_svr))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_svr))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE on the training dataset is: ', sqrt(mean_squared_error(y_train, y_pred_train_svr)))
print('The RMSE on the testing dataset is: ', sqrt(mean_squared_error(y_test, y_pred_test_svr)))
print("")

# The Mean Absolute Error (MAE)
print('The MAE on the training dataset is: ', mean_absolute_error(y_train, y_pred_train_svr))
print('The MAE on the testing dataset is: ', mean_absolute_error(y_test, y_pred_test_svr))

The R2 score on the training dataset prediction is:  0.9835668471166394
The R2 score on the test dataset prediction is:  0.9855753572146352

The RMSE on the training dataset is:  18.320981095847927
The RMSE on the testing dataset is:  17.365918592691642

The MAE on the training dataset is:  10.152824283809212
The MAE on the testing dataset is:  9.864461130426806


In [33]:
# Training with XGBoost regressor
xgb_regressor = XGBRegressor(objective='reg:squarederror')

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', xgb_regressor)])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict on the training and test datasets
y_pred_train_xgb = pipeline.predict(X_train)
y_pred_test_xgb = pipeline.predict(X_test)

In [34]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train, y_pred_train_xgb))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_xgb))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE on the training dataset is: ', sqrt(mean_squared_error(y_train, y_pred_train_xgb)))
print('The RMSE on the testing dataset is: ', sqrt(mean_squared_error(y_test, y_pred_test_xgb)))
print("")

# The Mean Absolute Error (MAE)
print('The MAE on the training dataset is: ', mean_absolute_error(y_train, y_pred_train_xgb))
print('The MAE on the testing dataset is: ', mean_absolute_error(y_test, y_pred_test_xgb))

The R2 score on the training dataset prediction is:  0.9999999995337128
The R2 score on the test dataset prediction is:  0.9999342113461649

The RMSE on the training dataset is:  0.003086135817686924
The RMSE on the testing dataset is:  1.1727912775247609

The MAE on the training dataset is:  0.0011593126877578203
The MAE on the testing dataset is:  0.16381329105746362


In [35]:
# training with neural network regressor

nn = MLPRegressor(hidden_layer_sizes=(8,), activation='relu', solver='adam', max_iter=2000, learning_rate_init=0.001)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', nn)])

# Fit the pipeline to the training data
pipeline.fit(X_train,y_train)

# Predict on the training and test datasets
y_pred_train_nn = pipeline.predict(X_train)
y_pred_test_nn = pipeline.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [36]:
# Model Accuracy on training dataset
print('The R2 score on the training dataset prediction is: ', r2_score(y_train, y_pred_train_nn))
print('The R2 score on the test dataset prediction is: ', r2_score(y_test, y_pred_test_nn))
print("")

# The Root Mean Squared Error (RMSE)
print('The RMSE on the training dataset is: ', sqrt(mean_squared_error(y_train, y_pred_train_nn)))
print('The RMSE on the testing dataset is: ', sqrt(mean_squared_error(y_test, y_pred_test_nn)))
print("")

# The Mean Absolute Error (MAE)
print('The MAE on the training dataset is: ', mean_absolute_error(y_train, y_pred_train_nn))
print('The MAE on the testing dataset is: ', mean_absolute_error(y_test, y_pred_test_nn))

The R2 score on the training dataset prediction is:  0.9954836110888766
The R2 score on the test dataset prediction is:  0.9956239367436461

The RMSE on the training dataset is:  9.604703110193784
The RMSE on the testing dataset is:  9.56504511719299

The MAE on the training dataset is:  6.302521225534942
The MAE on the testing dataset is:  6.32906225969804
